In [18]:
import poppy
import jax.numpy as jnp
from   jax       import random, jit, vmap
from functools  import partial

In [2]:
p = 5
n = 3
f = poppy.field( p, n )

In [3]:
d = 4
a = random.randint( random.PRNGKey( poppy.seed + 0 ), ( d, d ), 0, p ** n, dtype = jnp.int64 )
b = random.randint( random.PRNGKey( poppy.seed + 1 ), ( d, d ), 0, p ** n, dtype = jnp.int64 )

In [4]:
ap = poppy.array( a, f )
bp = poppy.array( b, f )

In [5]:
ap.shape, ap.rep.shape, ap.rep

((4, 4), (12, 12))

In [6]:
api = ap.inv( )
bpi = bp.inv( )

In [7]:
print( ap.rep[ 0, : 10 ] )
( ap - ap @ api @ ap @ bpi @ bp ).rep

[2 4 1 3 0 3 3 0 0 1]


Array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int64)

In [72]:
@partial( jit, static_argnums = ( 1, 2 ) )
def poppyhash( a, F, FF ):
    b = poppy.proj( a, F )
    c = poppy.proj( b, FF )
    return c

In [75]:
liftbatch = vmap( poppy.lift, in_axes = ( 0, None ) )
hashbatch = vmap( poppyhash,  in_axes = ( 0, None, None ) )

In [89]:
p  = 5
n  = 6
d  = 10
N  = 100000
a  = random.randint( random.PRNGKey( poppy.seed ), ( N, d, d ), 0, p ** n, dtype = jnp.int64 )
ap = liftbatch( a, poppy.field( p, n ) )
F  = poppy.field( 31, n )
FF = poppy.field( 7, d )
h  = hashbatch( ap, F, FF ).squeeze( )
au = jnp.unique( a, axis = 0 )
hu = jnp.unique( h )
len( au ), len( hu )

(100000, 99983)